## Imports

In [1]:
import sys
import pandas as pd
import numpy as np
import pickle

from functools import partial

import sklearn.pipeline as skpip
from sklearn.compose import ColumnTransformer
from category_encoders import WOEEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

# Import my libs
sys.path.append('../')

from utils.logger import get_logger
from src.features.balance import BalanceMixin
from src.features.correlation import HighCorrelation_filter
from src.features.variance import NearZeroVar_filter
from src.features.passthrough import Passthrough

modelisationTuple

In [2]:
with open('../data/processed/modelling_tuple', 'rb') as f:
    M = pickle.load(f)

## Preprocessing Steps

#### Function I use in preprocessing

In [3]:
def _intersect(df: pd.DataFrame, target: list):
    """
    Intersection between a list and a set of columns in a dataframe.

    Args:
        df (pd.DataFrame): The dataframe with which we want to make the intersection.
        target (list): The array with which we want to make the intersection.
    """

    return [c for c in df.columns if c in target]

####  Split the features to be processed by their types

In [4]:
#M.train_X.dtypes

In [5]:
# variables that do not require preprocessing
passthrough_cols = ['']

# non-ordinal / categorical variables
categorical_cols = [''] 

# quantitative variables
quantitative_cols = set(M.train_X.select_dtypes(include=['float','int']).columns.tolist()) - set(passthrough_cols) - set(categorical_cols)
quantitative_cols = list(quantitative_cols)

####  Fast checkpoint

In [6]:
# Check if dataframe contains NaN value
test = M.train_X.isna().sum().to_frame('nb')
test[test['nb'] > 0]

,nb


In [7]:
 # Check if dataframe contains infinity
test = np.isinf(M.train_X).sum().to_frame('nb')
test[test['nb'] > 0]

,nb


In [8]:
 # prevalence
y = M.train_y
out = pd.concat([
    np.round(y.value_counts(normalize=True, ascending=False).rename('normalized'), 4) * 100,
    y.value_counts(normalize=False, ascending=False).rename('number'),
], axis=1)
out

,normalized,number
Class,,
0.0,99.83,217497
1.0,0.17,380


#### Preprocessor

In [9]:
def build_data_preprocessor(passthrough_cols: list, categorical_cols: list, quantitative_cols: list, scale=False):
    """
    Creates a data preprocessing pipeline

    Args:
        passthrough_cols (list):  list of variables that do not require preprocessing.
        categorical_cols (list):  list of non-ordinal / categorical variables .
        quantitative_cols (list): list of quantitative variables.
        scale (boolean, optional): Should we standardize the variables? Defaults to False.
    """

    # This step will make each preprocessing step robust if it does not receive all the assumed variables
    passthrough_cols_selector = partial(_intersect, target=passthrough_cols)
    categorical_cols_selector = partial(_intersect, target=categorical_cols)
    quantitative_cols_selector = partial(_intersect, target=quantitative_cols)

    # Categorical-non-ordinal pipeline
    categorical_transformer = skpip.Pipeline(steps=[
        ('encoder', WOEEncoder())
    ])

    # Passthrough pipeline
    passthrough_transformer = skpip.Pipeline(steps=[
        ('passthrough', Passthrough())
    ])

    # Quantitative pipeline
    quantitative_transformer = skpip.Pipeline(steps=[
        ('NZV', NearZeroVar_filter(equisample=True)),
        ('high-correlation', HighCorrelation_filter(equisample=True)),
        ('imputer', SimpleImputer(strategy='median')),
    ])

    if scale:
        quantitative_transformer.steps.append(('scaler', StandardScaler()))


    # Map the columns to their respective transformers
    # ColumnTransformer allows to apply data transformations to different features of a same df. 
    # Columns not specified in the "Transformers" list are removed from the default dataset.
    columns_transformers = ColumnTransformer(
        transformers=[
            ('quantitative', quantitative_transformer, quantitative_cols_selector),
            ('passthrough', passthrough_transformer, passthrough_cols_selector),
            ('categorical', categorical_transformer, categorical_cols_selector),
        ],
        verbose_feature_names_out=False,
    )

    # Define the pipeline
    preprocessor = skpip.Pipeline(steps=[
        ('columns_transformers', columns_transformers),
    ]
    )

    return preprocessor

## SmokeTest

In [10]:
preprocessor = build_data_preprocessor(passthrough_cols= passthrough_cols, categorical_cols= categorical_cols, quantitative_cols= quantitative_cols, scale=False)
preprocessor.fit(M.train_X, M.train_y)

2023-09-25 15:18:09,753 [NearZeroVar_filter] INFO     Number of variables to delete : 0
2023-09-25 15:18:09,827 [HighCorrelation_filter] INFO     Number of variables to delete : 1


Pipeline(steps=[('columns_transformers',
                 ColumnTransformer(transformers=[('quantitative',
                                                  Pipeline(steps=[('NZV',
                                                                   NearZeroVar_filter()),
                                                                  ('high-correlation',
                                                                   HighCorrelation_filter()),
                                                                  ('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  functools.partial(<function _intersect at 0x7f97bf1e0700>, target=['V20', 'Amount', 'V25', 'V4', 'V28', 'V10', 'V19', 'V13', 'V21', 'V3',...', 'V8', 'V7', 'V5', 'V2', 'V11', 'Time', 'V15', 'V6', 'V18'])),
                                                 ('passthrough',
                                                  Pipeline(steps=[('passthrough',
                                                                   Passthrough())]),
                                                  functools.partial(<function _intersect at 0x7f97bf1e0700>, target=[''])),
                                                 ('categorical',
                                                  Pipeline(steps=[('encoder',
                                                                   WOEEncoder())]),
                                                  functools.partial(<function _intersect at 0x7f97bf1e0700>, target=['']))],
                                   verbose_feature_names_out=False))])

In [11]:
print(type(preprocessor))
print(type(preprocessor.named_steps['columns_transformers']))

<class 'sklearn.pipeline.Pipeline'>
<class 'sklearn.compose._column_transformer.ColumnTransformer'>


In [12]:
# Extract the initial columns names from the design matrix
preprocessor.feature_names_in_

array(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9',
       'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18',
       'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27',
       'V28', 'Amount'], dtype=object)

In [13]:
# Extract the columns names from the FINAL layer of the transformer
preprocessor.get_feature_names_out()

array(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9',
       'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V18', 'V19',
       'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28',
       'Amount'], dtype=object)

In [14]:
X_transformed = preprocessor.transform(M.train_X)
X_transformed = pd.DataFrame(X_transformed, columns=preprocessor.get_feature_names_out())